# Duomenų tvarkymas

2020-09-29

- Failų ir direktorijų struktūra
- Python direktorijų ir failų tvarkymo bibliotekos ir darbas su jomis
  - Pathlib, os
  - csv, json, txt
- Paieška: Regex
- Klaidų taisymas

Pirmi žingsniai pradedant duomenų analizę:

- Surasti reikiamus duomenų masyvus
- Importuoti duomenis

## Bendra tvarka

- Nenaudoti failų ir direktorijų varduose šių simbolių:
  - LT ar pan. raidžių
  - skyrybos ženklų (, ? ! ir t.t.)
- Naudoti: (a-z A-Z 0-9 \_)
- Strategijos:
  - Pirmas raides žodžių didžiosiomis raidėmis - VienasDu (CamelCase)
  - Atskirt žodžius apatiniais brūkšniais vienas\_du (snake case)

## Direktorijų struktūra

### Direktorijų medis

Unix kompiuteriuose direktorijas skiria pasvirasis brūkšnys `/`

Windows operacinėse sistemose atgalinis brūkšnys `\`

### Rekomenduojama neuromokslų struktūra: bids.neuroimaging.io

![bids](../images/bids.png)

### Dažniausiai aptinkama struktūra git projektuose

Failas | Paskirtis
---|---
README.md | Aprašas projekto
LICENSE | choosealicense.com
setup.py | Distribucijos valdymas
requirements.txt | programos reikalavimai
sample/\_\_init\_\_.py | python bibliotekos
sample/core.py | Pagrindinė programa
docs/conf.py | Dokumentacija
docs/index.md | Indeksas
tests/test\_basic.py | Testai

### Minimali struktūra

Failas ar direktorija | Paskirtis
---|---
Code/ | Direktorija skirta skaičiavimų kodui
Data/ | Direktorija skirta duomenims
Text/ | Direktorija skirta tekstu
Text/README.md | Aprašas
Text/Figures/ | Direktorija skirta paveikslams
.gitignore | versijos kontrolė
.git/ | versijos kontrolė

### Failų tipai

- .txt
- .csv
- .xlsx
- .json
- .mat
- .pdf
- .html
- .dat
- t.t.

## Tvarkymas direktorijų ir failų su python

Dažniausiai direktorijų tvarkymas atliekamas keturiomis pakuotėmis:

- os --- operacinės sistemos rutinos
- glob --- paieškos funkcijos
- shutil --- įvairios funkcijos kopijavimui ir pan.
- pathilb  --- nuo python versijos 3.4 atsiradusi `pathlib` pakuotė
gali viena atlikti daugumą direktorijų ir failų tvarkymo veiksmų.

In [1]:
import re
import glob
import os
import shutil
import pathlib

### `os`

Dažnai pasitaikanti pakuotė programos sąveikai su operacine sistema valdyti.

Norint gauti namų direktoriją nepriklausomai nuo sistemos:

In [2]:
home_dir = os.path.expanduser("~")
home_dir

'/home/aleks'

Jei norime dokumentuose sukurti direktoriją `biologija` negerai būtų ją
aprašyti kaip:

In [3]:
direktorija = '/home/neuro/Documents' + '/biologija'
direktorija

'/home/neuro/Documents/biologija'

arba

In [4]:
direktorija = os.path.expanduser("~") + '/Documents/biologija'
direktorija

'/home/aleks/Documents/biologija'

Kuriant direktorijų pavadinimus svarbu kad jie būtų sukuriami nepriklausomai nuo
sistemos kur bus naudojama programa.

In [5]:
test_dir = os.path.join(home_dir, "Documents", "biod2020", "datatest")
test_file = os.path.join(home_dir, "Documents", "biod2020", "datatest", "test.txt")
test_dir, test_file

('/home/aleks/Documents/biod2020/datatest',
 '/home/aleks/Documents/biod2020/datatest/test.txt')

Galime sukurti aprašytą failą nusiųsdami komandą operacinei sistemai:

In [6]:
os.system(f"mkdir {test_dir}")
os.system(f"touch {test_file}")

0

Arba `os` metodais

In [7]:
os.mkdir(test_dir)

FileExistsError: [Errno 17] File exists: '/home/aleks/Documents/biod2020/datatest'

Visą medį direktorijų sukuriam

In [8]:
os.makedirs("test/test1/test2", exist_ok=True)
!ls test

test1


Pakeisti dabartinę direktoriją

In [9]:
os.chdir(test_dir)
%pwd

'/home/aleks/Documents/biod2020/datatest'

In [10]:
pwd

'/home/aleks/Documents/biod2020/datatest'

Sužinom dabartinę direktoriją

In [11]:
current_working_directory = os.getcwd()
current_working_directory

'/home/aleks/Documents/biod2020/datatest'

Gaunam viduje esančių failų struktūrą

In [12]:
ls

test.txt


In [13]:
dfiles = os.listdir()
dfiles

['test.txt']

Pakeičiam pavadinimus

In [14]:
test_file2 = os.path.join(home_dir, "Documents",
                          "biod2020", "datatest", "test2.txt")
os.rename(test_file, test_file2)

Ištrinam failus

In [15]:
if os.path.isfile(test_file2):
    os.remove(test_file2)
    # os.unlink(file)
else:
    print(f"file {file} does not exist")

Ištrinam prieš tai sukurtą direktorijų medį

In [16]:
os.chdir('../lect')
for dirpath, dirnames, files in os.walk("./test", topdown=False):
    try:
        os.rmdir(dirpath)
        #  shutil.rmtree()
    except OSError as ex:
        print(f"Error: {dirpath} : {e.strerror}")

Eiti per direktorijas ir sub direktorijas galime funkcija `walk`.

Einame direktorijų medžiu ir ieškome pdf failų.

In [17]:
bioa_dir = os.path.join(home_dir, "Documents", "biod2020")
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(bioa_dir, topdown=True):
    #   print(r, f, d)
    for file in f:
        if ".pdf" in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

/home/aleks/Documents/biod2020/lab/labreport.nbconvert.pdf
/home/aleks/Documents/biod2020/lab/lab1.pdf
/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab1-checkpoint.pdf
/home/aleks/Documents/biod2020/images/EEGsources.pdf
/home/aleks/Documents/biod2020/images/ERPmine.pdf
/home/aleks/Documents/biod2020/lect/lect1_Intro.pdf
/home/aleks/Documents/biod2020/lect/.ipynb_checkpoints/lect1_Intro-checkpoint.pdf


Plėtinius galima ieškoti ir su `endswith` metodu

In [18]:
for r, d, f in os.walk("./"):
    for file in f:
        if file.endswith(".pdf"):  # startswith
            print(file)

lect1_Intro.pdf
lect1_Intro-checkpoint.pdf


### glob

Paprastesnis būdas ieškoti failų su glob pakote

In [19]:
os.chdir(bioa_dir)
glob.glob("**/*.pdf", recursive=True)

['lab/labreport.nbconvert.pdf',
 'lab/lab1.pdf',
 'images/EEGsources.pdf',
 'images/ERPmine.pdf',
 'lect/lect1_Intro.pdf']

\*\* nurodo ieškoti ir subdirektorijose

rglob metodas ieškotu subdirektorijose ir be dviejų žvaigždučių

Glob paieškos specifiniai simboliai:

Simbolis | Reikšmė
---|---
\* | pakeičiamas į 0 arba daugiau simbolių (l*.pdf surastų tiek lect1.pdf tiek ir lab1.pdf)
\? |  pakeičiamas į vieną betkokį simbolį (l??.pdf sureas tik lab.pdf)
[seq] | surandą simbolius nurodytus tarp skliaustų ([0-9] [a-z] ieškos simbolių šiuose atkarpose)
[!seq] | atvirkštinis variantas ieškos simbolių nesančių sąraše

In [20]:
for name in glob.glob("**/*[1-9]*.ipynb", recursive=True):
    print(name)

lab/lab2.ipynb
lab/lab1.ipynb
lab/lab3.ipynb
lect/lect2_Python.ipynb
lect/lect1_Intro.ipynb
lect/lect3_Path.ipynb


>> **UŽDUOTIS**
>>
>> Raskite visus py failus
>>
---
```

### shutil

Kopijavimas

In [ ]:
os.system(f"touch {test_file}")
shutil.copy(test_file, test_file2)  # same as cp in shell, preserves metadata

In [ ]:
shutil.copytree(source_directory, destination_directory)

In [ ]:
shutil.move(souce, destination)  # files or directories

In [ ]:
help(os)

In [ ]:
help(glob)

In [ ]:
help(shutil)

### pathlib

Pathlib pakuotė sukuria objektus reprezentuojančius direktorijas ir failus.

Tada su pathlib metodais mes modifikuojame sukurtus objektus.

In [21]:
test_file_pathlib = pathlib.Path(test_file)
test_dir_pathlib = pathlib.Path(test_dir)
test_file_pathlib, test_dir_pathlib

(PosixPath('/home/aleks/Documents/biod2020/datatest/test.txt'),
 PosixPath('/home/aleks/Documents/biod2020/datatest'))

Dabartinė direktorija

In [22]:
p = pathlib.Path.cwd()
p

PosixPath('/home/aleks/Documents/biod2020')

Namų direktorija

In [23]:
p = pathlib.Path.home()
p

PosixPath('/home/aleks')

Kūrimas universalių nuorodų į failus ir direktorijas

In [24]:
p = pathlib.Path.joinpath(pathlib.Path.home(), "Documents", "biod2020")
p

PosixPath('/home/aleks/Documents/biod2020')

Tikrinimas ar direktorija/failas egzistuoja

In [25]:
p.exists()

True

Tikrinimas ar tai direktorija

In [26]:
p.is_dir()

True

Tikrinimas ar tai failas

In [27]:
p.is_file()

False

Kūrimas direktorijų

In [ ]:
test_dir_pathlib.mkdir(parents=True, exist_ok=True)

Sukūrimas failo pagal objektą

In [28]:
test_file_pathlib.touch()

Keitimas vardo

In [29]:
test_file_pathlib = test_file_pathlib.rename('test2.txt')

Ištrynimas direktorijos

In [30]:
test_dir_pathlib.rmdir()

Ištrynimas failo

In [31]:
test_file_pathlib.unlink()

Paieška failų ir direktorijų

In [32]:
r = p.glob("**/*.pdf")
list(r)

[PosixPath('/home/aleks/Documents/biod2020/lab/labreport.nbconvert.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/lab/lab1.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab1-checkpoint.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/images/EEGsources.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/images/ERPmine.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/lect/lect1_Intro.pdf'),
 PosixPath('/home/aleks/Documents/biod2020/lect/.ipynb_checkpoints/lect1_Intro-checkpoint.pdf')]

Plačiau

In [33]:
help(pathlib.Path)

Help on class Path in module pathlib:

class Path(PurePath)
 |  Path(*args, **kwargs)
 |  
 |  PurePath subclass that can make system calls.
 |  
 |  Path represents a filesystem path but unlike PurePath, also offers
 |  methods to do system calls on path objects. Depending on your system,
 |  instantiating a Path will return either a PosixPath or a WindowsPath
 |  object. You can also instantiate a PosixPath or WindowsPath directly,
 |  but cannot instantiate a WindowsPath on a POSIX system or vice versa.
 |  
 |  Method resolution order:
 |      Path
 |      PurePath
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, t, v, tb)
 |  
 |  absolute(self)
 |      Return an absolute version of this path.  This function works
 |      even if the path doesn't point to anything.
 |      
 |      No normalization is done, i.e. all '.' and '..' will be kept along.
 |      Use resolve() to get the canonical path to a file.
 |  
 |  chmod(self,

---
>> **UŽDUOTIS**
>>
>> - Naudodami pathlib sukurkite savo direktorijų medį
>> - Naudodami pathlib raskite visus ipynb failus
>>
---

## Tekstiniai failai

Sukuriam failą

In [34]:
test_file_pathlib.touch()

Atidarome failą rašymo režime `w` ir įrašyti žodį 'text'

In [35]:
failas = open(test_file_pathlib, "w")
try:
    failas.write("text")
finally:
    failas.close()

Komanda | Reikšmė
---|---
'r' | skaitymas (default)
'w' | rašymas, pirma ištrinant
'x' | sukurt naują failą ir įrašyti
'a' | rašymas, pridėti tekstą
'b' | dvejetainis rėžimas

Kad neužmirštume uždaryti failo naudojame konteksto tvarkyklę `with`

Atidarome failą skaitymo rėžime `r` ir nuskaitome visas eilutes

In [36]:
with open(test_file_pathlib, "r") as failas:
    tt = failas.readlines()  # list su visais simboliais
    print(tt)

['text']


Norėdami skaityti po vieną eilutę

In [37]:
with open(test_file_pathlib, "r") as failas:
    for row in failas:
        print(row, end="")

text

Pathlib pakuotė turi metodus nuskaityti ar įrašyti tekstą į savo objektus.

In [38]:
test_file_pathlib.read_text()

'text'

In [39]:
test_file_pathlib.write_text("abc")

3

Apjungiant susimuiluojame eksperimento failus.

In [40]:
test_dir_pathlib.mkdir()
for subj in range(1, 11):
    for txt_file in range(1, 11):
        if txt_file < 6:
            measure = "power"
        else:
            measure = "phase"
        p = pathlib.Path.joinpath(
            pathlib.Path.home(),
            "Documents",
            "biod2020",
            "datatest",
            f"Exp_{txt_file}_{measure}_{subj}.txt",
        )
        p.touch()

Ištriname direktorijas sukurtas

In [41]:
test_file_pathlib.unlink()


def rm_tree(pth):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()


rm_tree(test_dir_pathlib)

## Regex

Kartais paprastų filtrų neužtenka tiek norint surasti informaciją.

Regex kalba yra universalus būdas (veikiantis visuose programavimo kalbose)
automatizuoti informacijos filtravimą.

Aptarsime tik mažą dalį šios kalbos, plačiau aprašyta
[dokumentacijoje](https://docs.python.org/3/howto/regex.html)

regex sakinių testavimas https://regex101.com/

### Kur galima naudoti regex

Dauguma duomenų analizės uždavinių galima apibūdinti kaip fragmentų paiešką
tekste.

- Radimas failų pavadinimų
- Radimas specifinių duomenų faile
- Analizė teksto sekų (DNR)
- Ištraukimas koordinačių
- Taksonominių vardų
- t.t.

Pavyzdys [regex DNR analizėje](https://pythonforbiologists.com/regular-expressions)

Python pakuotė `re` aprašo regex metodus

### Failų radimas

In [42]:
def find_files_glob(directory, pattern):
    return list(Path(directory).rglob(pattern))


file_list = find_files_glob(bioa_dir, "*py")
file_list

[PosixPath('/home/aleks/Documents/biod2020/lab/lab2.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/programa.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/lab3.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/labreport.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/programa-checkpoint.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab2-checkpoint.py'),
 PosixPath('/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab3-checkpoint.py'),
 PosixPath('/home/aleks/Documents/biod2020/lect/programa2.py'),
 PosixPath('/home/aleks/Documents/biod2020/lect/lect2_Python.py'),
 PosixPath('/home/aleks/Documents/biod2020/lect/lect1_Intro.py'),
 PosixPath('/home/aleks/Documents/biod2020/lect/.ipynb_checkpoints/lect2_Python-checkpoint.py'),
 PosixPath('/home/aleks/Documents/biod2020/lect/.ipynb_checkpoints/programa2-checkpoint.py')]

Jei mums reikia sudėtingesnės logikos pvz jei norime rasti tik tuos py failus
kurie yra lab arba exam direktorijose.

In [43]:
def find_files_re(directory, pattern):
    # in first for loop we convert to strings pathlib returned directory contents
    files = [str(element) for element in list(Path(directory).rglob("./*"))]
    # in second loop using regular expressions we filter pathlib output
    files = [element for element in files if re.search(pattern, element)]
    return files


file_list = find_files_re(bioa_dir, r".*(exam|lab).*py$")
file_list

['/home/aleks/Documents/biod2020/lab/lab2.py',
 '/home/aleks/Documents/biod2020/lab/programa.py',
 '/home/aleks/Documents/biod2020/lab/lab3.py',
 '/home/aleks/Documents/biod2020/lab/labreport.py',
 '/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/programa-checkpoint.py',
 '/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab2-checkpoint.py',
 '/home/aleks/Documents/biod2020/lab/.ipynb_checkpoints/lab3-checkpoint.py']

Pradžioje apžvelgiam regex sakinių konstrukciją ir tada `re` pakuotės metodus.

Kadangi yra daug specialių simbolių formuojant regex sakinius pravartu naudoti
r-strings.

Simboliai | Reikšmė
---|---
\. | Bet koks simbolis išskyrus nauja eilutė
\* | 0 arba daugiau kartų pasikartojanti prieš tai einanti komanda
\+ | 1 arba daugiau kartų
\? | 0 arba 1 kartą
[a] | specifinis simbolis
[a-z] | raidės nuo a iki z
[a-zA-Z0-9\_] | mažosios arba didžiosios raidės, skaičiai ir \_
^ | Jei naudojamas kaip pirmas simbolis tai simbolizuoja eilutės pradžią
[^A] | ne A
$ | eilutės pabaiga
{x,n} | komanda arba simbolis pakartojamas nuo x iki n kartų
(x,x1,..) | grupuojami simboiai
\| | jungtukas arba
\d | skaičius nuo 0 iki 9
\D | tas pats kaip [^0-9]
\s |  tarpas
\S |  ne tarpas
\W | tas pats kaip [^a-zA-Z0-9\_]
\w | tas pats kaip [a-zA-Z0-9\_]
\b | žodžio kraštas
\B | ne kraštas
\  | leidžia naudoti simbolius rezervuotus komandoms
\n | nauja eilutė

#### re metodai

Jei turime kintamąjį HI

In [44]:
HI = "Hello 1 5 world! 5"

Ir norime surasti visus skaičius šiame tekste

In [45]:
re.findall(r"\d+", HI)

['1', '5', '5']

Dažniausiai pradedame kurti paieška įtraukdami viską --- . komanda

In [46]:
re.match(r".*", HI)

<re.Match object; span=(0, 18), match='Hello 1 5 world! 5'>

Metodas `group` gražina aptiktą reikšmę

In [47]:
re.match(r".*", HI).group()

'Hello 1 5 world! 5'

In [48]:
re.findall(r"\d+", HI)

['1', '5', '5']

'\d+' surask bet kokį skaičių `\d` pakartotą `+` 1 arba daugiau kartų

finditer gražina po vieną reikšmes for ciklui

In [49]:
list(re.finditer(r"\d+", HI))

[<re.Match object; span=(6, 7), match='1'>,
 <re.Match object; span=(8, 9), match='5'>,
 <re.Match object; span=(17, 18), match='5'>]

`re.search` -  skenuoja tekstą ieškodamas atitinkančio šablono.
Jis panašus į match metodą bet gražina ne vien tik pirmą rastą reikšmę.

Raskime skaičių 5 kuris yra teksto pabaigoje

In [50]:
re.search(r"5$", HI).group()

'5'

Metodas span gražina simbolių vietą tekste

In [51]:
re.search(r"5$", HI).span()

(17, 18)

`5` randą skaičių o $ teksto pabaigą

match panašia komanda neranda nieko nes skenuoja nuo pradžių tekstą

In [52]:
re.match(r"5$", HI)

`e.split` padalina ties rastais simboliais tekstą į dalis

In [54]:
re.split(r"\s", HI)

['Hello', '1', '5', 'world!', '5']

`\s` randa tarpus tarp žodžių

Padalinam tekstą ties skaičiais

In [55]:
re.split(r"\d", HI)

['Hello ', ' ', ' world! ', '']

Matome kad skaičiai išnyksta nes ties jais vyksta dalinimas.
Jei norime išlaikyti skaičius juos reik sugauti ()

In [56]:
re.split(r"(\d)", HI)

['Hello ', '1', ' ', '5', ' world! ', '5', '']

`re.sub` pakeičia į kitas reikšmes

In [57]:
re.sub(r"!", "", HI)

'Hello 1 5 world 5'

Rasti žodžius po kurių seka šauktukas galima su lookahead sintakse (?=x)

In [58]:
re.findall(r"\w+(?=!)", HI)

['world']

Ir atvirkštinis variantas jei pavyzdžiui nenorime rasti plėtinių pdf arba ipynb

In [59]:
file_list = find_files_re(bioa_dir, r'.*[.](?!pdf$|ipynb$)[^.]*$')
file_list

['/home/aleks/Documents/biod2020/planas.docx',
 '/home/aleks/Documents/biod2020/.git',
 '/home/aleks/Documents/biod2020/.gitignore',
 '/home/aleks/Documents/biod2020/README.md',
 '/home/aleks/Documents/biod2020/.ipynb_checkpoints',
 '/home/aleks/Documents/biod2020/.git/objects',
 '/home/aleks/Documents/biod2020/.git/config',
 '/home/aleks/Documents/biod2020/.git/COMMIT_EDITMSG',
 '/home/aleks/Documents/biod2020/.git/FETCH_HEAD',
 '/home/aleks/Documents/biod2020/.git/branches',
 '/home/aleks/Documents/biod2020/.git/hooks',
 '/home/aleks/Documents/biod2020/.git/refs',
 '/home/aleks/Documents/biod2020/.git/logs',
 '/home/aleks/Documents/biod2020/.git/description',
 '/home/aleks/Documents/biod2020/.git/ORIG_HEAD',
 '/home/aleks/Documents/biod2020/.git/HEAD',
 '/home/aleks/Documents/biod2020/.git/info',
 '/home/aleks/Documents/biod2020/.git/index',
 '/home/aleks/Documents/biod2020/.git/objects/c4',
 '/home/aleks/Documents/biod2020/.git/objects/0c',
 '/home/aleks/Documents/biod2020/.git/obje

[dokumentacija](https://docs.python.org/3/howto/regex.html)

## Atpažinimas ir ištaisymas klaidų (debugging)

- Bandyt reikia išvengti klaidų ir jų paieškos! :)
- Neįmanoma parašyti kodo be klaidų
- Yra daug įrankių priklausomai nuo užduočių

### Kodo skaitymas

- Kodas yra vykdomas iš viršaus į apačią.
- Importuojami failai paleidžia kitus failus

In [60]:
import math
for item in list(range(10)):
    print(item)

0
1
2
3
4
5
6
7
8
9


### Skaitymas klaidos pranešimo (tracebacks)

In [61]:
from pathlib import Path
path = Path('tt.txt')
txt = path.read_text()

FileNotFoundError: [Errno 2] No such file or directory: 'tt.txt'

- Dažniausiai klaida bus mūsų kode o ne bibliotekoje.
- Paskutinė žinutė praneša apie klaidos tipą

Galimos žinutės ir tipai:

In [62]:
5 ==== 6

SyntaxError: invalid syntax (<ipython-input-62-9a0e44ab5405>, line 1)

In [63]:
def fun_(a):
return a

IndentationError: expected an indented block (<ipython-input-63-0a4276a0b1dc>, line 2)

In [65]:
prrrint('aaa')

NameError: name 'prrrint' is not defined

In [66]:
a = [2,3,4]
a.te()

AttributeError: 'list' object has no attribute 'te'

In [67]:
a[8]

IndexError: list index out of range

Python dokumentacijoje aprašyti visi tipai

### Print

- Dažniausiai pasitaikantis ir lengviausias būdas
- Gerai tinka logikai suprasti kodo, trumpiems algoritmams
- Bet taip pat lėčiausias ir galintis įvelti klaidų.

In [68]:
for n in range(2, 10):
    print(f"n {n}", flush=True)
    for x in range(2, n):
        print(f"x {x}", flush=True)
        if n % x == 0:
            print(n, "equals", x, "*", n // x)
            break
    else: 
        print(n, "is a prime number")

n 2
2 is a prime number
n 3
x 2
3 is a prime number
n 4
x 2
4 equals 2 * 2
n 5
x 2
x 3
x 4
5 is a prime number
n 6
x 2
6 equals 2 * 3
n 7
x 2
x 3
x 4
x 5
x 6
7 is a prime number
n 8
x 2
8 equals 2 * 4
n 9
x 2
x 3
9 equals 3 * 3


### Python debugger (PDB)

- Standard library debugger
- Iš komandinės eilutės paleidžiamas ir galima po žingsnį eiti
- python -m pdb script.py
- arba kode įterpti breakpoint()
- h help
- q quit
- ll dabartinė pozicija kode
- n next
- s step
- c continue
- etc

https://docs.python.org/3.6/library/pdb.html

In [69]:
def prime_detector():
    nonprime=[]
    primes=[]
    for value in range(2, 10):
        for value2 in range(2, value):
            breakpoint()
            if value % value2 == 0:
                nonprime.append(value)
                break
        else:
            primes.append(value)

In [70]:
prime_detector()

> <ipython-input-69-b993a50546b8>(7)prime_detector()
      5         for value2 in range(2, value):
      6             breakpoint()
----> 7             if value % value2 == 0:
      8                 nonprime.append(value)
      9                 break



ipdb>  ll


      1 def prime_detector():
      2     nonprime=[]
      3     primes=[]
      4     for value in range(2, 10):
      5         for value2 in range(2, value):
      6             breakpoint()
----> 7             if value % value2 == 0:
      8                 nonprime.append(value)
      9                 break
     10         else:
     11             primes.append(value)



ipdb>  primes


[2]


...

> <ipython-input-69-b993a50546b8>(5)prime_detector()
      3     primes=[]
      4     for value in range(2, 10):
----> 5         for value2 in range(2, value):
      6             breakpoint()
      7             if value % value2 == 0:



ipdb>  value


3


ipdb>  q


BdbQuit: 

### Jupyter

- %run script
- %debug post-mortem įeina į klaidų paieškos terpę
- %pdb on  --- auto enter ipdb on error

In [71]:
%pdb on
def prime_detector():
    nonprime=[]
    primes=[]
    for value in range(2, 10):
        for value2 in range(2, value):
            if vaue % value2 == 0:
                nonprime.append(value)
                break
        else:
            primes.append(value)

prime_detector()

Automatic pdb calling has been turned ON


NameError: name 'vaue' is not defined

> <ipython-input-71-f8f33bcef53b>(7)prime_detector()
      5     for value in range(2, 10):
      6         for value2 in range(2, value):
----> 7             if vaue % value2 == 0:
      8                 nonprime.append(value)
      9                 break



ipdb>  q


Kai nebereikia nepamiršti išjungti saugant resursus

In [72]:
%pdb off

Automatic pdb calling has been turned OFF


### Pycharm, VSCODE ir kiti

In [ ]:
from IPython.display import YouTubeVideo
video = YouTubeVideo(id='nksiGORLDZw', width=400, height=200, fs=1, autoplay=0)
video